**Group 1 DS & ML Final Project - Student Dropout Prediction**

**Project Overview**

- Build a predictive model to identify students at risk of dropping out (binary classification - Dropout vs. Graduate) using academic, demographic, and socioeconomic data.

**Team Members & Contributions**

- Member 1 (Imienu Charity): Project setup, data loading, data describtion, and GitHub repository management

- Member 2 (Abigaba Prosper): Data exploration, visualization, and statistical analysis

- Member 3 (Rwotolara Innocent): Data preprocessing, feature engineering, and PCA implementation

- Member 4 (Kiwanuka Kenneth): Classical ML models, hyperparameter tuning, and ensemble methods

- Member 5 (Ainembabazi Allan): Neural network development, training, and deep learning optimization

- Member 6 (Ekou David): Model evaluation, interpretation, business insights, and presentation

GitHub Repository: [Refractory_Machine-Learning](https://github.com/kossi-26/Refractory_Machine-Learning/)

In [ ]:
# Project Setup - Imienu Charity
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report, roc_curve, auc
import joblib
import warnings
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE

warnings.filterwarnings('ignore')
np.random.seed(42)
sns.set_theme()

In [ ]:
# Data Loading - Imienu Charity
# Load the primary UCI dataset
df = pd.read_csv('data.csv', delimiter=';')
print(f'Original UCI data shape: {df.shape}')

In [ ]:
# Feature Engineering - Rwotolara Innocent
# Add simulated Tanzania-specific features
np.random.seed(42)  # For reproducibility

# Rural/Urban divide
df['Location_Type'] = np.random.choice(['Urban', 'Rural'], size=len(df), p=[0.7, 0.3])

# Internet access (lower in rural areas)
df['Internet_Access'] = np.where(
    df['Location_Type'] == 'Urban',
    np.clip(np.random.normal(0.6, 0.2, len(df)), 0.1, 1.0),
    np.clip(np.random.normal(0.3, 0.15, len(df)), 0.05, 0.8)
).round(3)

# Teacher qualifications (lower in rural areas)
df['Teacher_Quality'] = np.where(
    df['Location_Type'] == 'Urban',
    np.clip(np.random.normal(0.75, 0.15, len(df)), 0.4, 1.0),
    np.clip(np.random.normal(0.5, 0.2, len(df)), 0.2, 0.9)
).round(3)

# Distance to school (greater in rural areas)
df['School_Distance_km'] = np.where(
    df['Location_Type'] == 'Urban',
    np.clip(np.random.gamma(2, 0.5, len(df)), 0.5, 5),
    np.clip(np.random.gamma(3, 2, len(df)), 1, 15)
).round(3)

print(f'Enhanced dataset shape: {df.shape}') # New features added - Location_Type, Internet_Access, Teacher_Quality, School_Distance_km
# df.head()

In [ ]:
# Data Description - Imienu Charity
# Show basic stats of the enhanced dataset
df.describe().round(3)

## Data Dictionary: Enhanced Student Dropout Prediction Dataset
**Source:** UCI ML Repository + Simulated Specific Features

**Citation:** Realinho, V., Vieira Martins, M., Machado, J., & Baptista, L. (2021)  

**DOI:** https://doi.org/10.24432/C5MC89  

**Students:** 4,424 | **Features:** 36 original + 4 simulated + 1 Target

## Target Variable
| Variable | Values | Description |
|----------|---------|-------------|
| **Target** | 0, 1 | 0=Dropout, 1=Graduate (filtered from original 3-class target) |

## New Simulated Features (Tanzania Context)
| Variable | Type | Values/Range | Description |
|----------|------|--------------|-------------|
| **Location_Type** | Categorical | Urban, Rural | School location type (70% Urban, 30% Rural) |
| **Internet_Access** | Numerical | 0.0-1.0 | Internet access quality score (0.0=None, 1.0=Perfect) |
| **Teacher_Quality** | Numerical | 0.0-1.0 | Teacher qualification score (0.0=Poor, 1.0=Excellent) |
| **School_Distance_km** | Numerical | 0.5-15 km | Distance from home to school in kilometers |

## Original Key Variables
- **Demographics:** Age at enrollment, Gender, Marital status, Nationality

- **Academic Background:** Previous qualification, Admission grade, Course

- **Academic Performance:** Curricular units approved/not approved, Grades

- **Financial:** Tuition fees up to date, Scholarship holder

- **Economic Context:** GDP, Unemployment rate

- **Family Background:** Mother's/Father's qualification and occupation

In [ ]:
# Data Preprocessing - Rwotolara Innocent
# Filter to binary targets for modeling
df = df[df['Target'].isin(['Graduate', 'Dropout'])].copy()
df['y'] = df['Target'].map({'Dropout': 0, 'Graduate': 1})
X = df.drop(columns=['Target', 'y'])
y = df['y']

In [ ]:
# Data Exploration and Visualization - Abigaba Prosper
# Pie chart for Target distribution (Overview of outcomes)
plt.figure(figsize=(6, 6))
df['Target'].value_counts().plot.pie(startangle=90,  autopct='%1.1f%%', colors=['#1f77b4', '#ff7f0e'], textprops={'fontsize': 14})
plt.title('Student Outcomes at a Glance', fontsize=18, fontweight='bold')
plt.ylabel('')
plt.savefig('target_pie.png', bbox_inches='tight', dpi=150)
plt.show()

Interpretation (Univariate) - This pie chart shows the distribution of student outcomes, with the majority being graduates, indicating a class imbalance where dropouts are less frequent but critical to predict.

In [ ]:
# Age bar plot - Abigaba Prosper
plt.figure(figsize=(8, 5))
sns.histplot(df['Age at enrollment'], bins=6, color='skyblue', kde=False, edgecolor='black')
plt.title('Age Distribution', fontsize=18, fontweight='bold')
plt.xlabel('Age', fontsize=14)
plt.ylabel('Number of Students', fontsize=14)
plt.savefig('age_bar.png', bbox_inches='tight', dpi=150)
plt.show()

Interpretation (Univariate) - The age distribution is right-skewed, with most students in their 20s, suggesting that many are recent high school graduates. Older students are less common, which may reflect different enrollment or dropout patterns.

In [ ]:
# Admission grade bar plot - Abigaba Prosper
plt.figure(figsize=(8, 5))
sns.histplot(df['Admission grade'], bins=15, kde=False, color='lightgreen', edgecolor='black')
plt.title('Admission Grades', fontsize=18, fontweight='bold')
plt.xlabel('Grade', fontsize=14)
plt.ylabel('Number of Students', fontsize=14)
plt.savefig('admission_bar.png', bbox_inches='tight', dpi=150)
plt.show()

Interpretation (Univariate) - Admission grades follow a normal distribution centered around 120-140, with fewer low scores, implying the dataset captures relatively high-performing students overall.

In [ ]:
# Bar plot for mean Admission grade by Target - Abigaba Prosper
plt.figure(figsize=(8, 5))
sns.barplot(x='Target', y='Admission grade', data=df, palette='Set2', errorbar=None)  # keep default CI or set errorbar='sd' for std
plt.title('Admission Grades vs. Graduation', fontsize=18, fontweight='bold')
plt.xlabel('Graduation Outcome', fontsize=14)
plt.ylabel('Average Admission Grade', fontsize=14)
plt.savefig('admission_by_target_bar.png', bbox_inches='tight', dpi=150)
plt.show()

Interpretation (Bivariate) - Both dropouts and graduates have relatively high admission grades, with only a small difference, suggesting that while academic performance matters, other non-academic factors also play a significant role in determining graduation outcomes

In [ ]:
# Stacked bar for School Distance by Location Type - Abigaba Prosper
plt.figure(figsize=(8, 5))
sns.histplot(data=df, x='School_Distance_km', hue='Location_Type', multiple='stack', palette=['#1f77b4', '#ff7f0e'], bins=10, edgecolor='black', alpha=0.8  )
plt.title('School Travel Distance by Location', fontsize=18, fontweight='bold')
plt.xlabel('Distance to School (km)', fontsize=14)
plt.ylabel('Number of Students', fontsize=14)
plt.savefig('school_distance_stacked.png', bbox_inches='tight', dpi=150)
plt.show()

Interpretation (Multivariate) - Rural students generally travel farther to school compared to urban students, which may increase the risk of dropouts due to accessibility challenges.

In [ ]:
# Bar plot for mean Internet Access by Target - Abigaba Prosper
df_filtered = df[df['Target'].isin(['Dropout', 'Graduate'])].copy()
plt.figure(figsize=(8, 5))
sns.barplot(x='Target', y='Internet_Access', data=df_filtered, palette='Set1', errorbar=None)
plt.title('Internet Access by Graduation Outcome', fontsize=18, fontweight='bold')
plt.xlabel('Graduation Outcome', fontsize=14)
plt.ylabel('Average Internet Access Score', fontsize=14)
plt.tight_layout()
plt.savefig('internet_by_target_bar.png', bbox_inches='tight', dpi=150)
plt.show()

Interpretation (Bivariate) - Graduation outcomes show little variation in internet access scores, indicating that factors beyond connectivity are likely more influential.

In [ ]:
# Horizontal bar plot of top correlations with Target - Abigaba Prosper
num_df = df.select_dtypes(include=['float64', 'int64'])
corr_with_target = num_df.corr()['y'].abs().drop('y').sort_values(ascending=False)
top_corr = corr_with_target.head(10)

plt.figure(figsize=(10, 6))
top_corr.sort_values().plot(kind='barh', color=sns.color_palette('bright', len(top_corr)))
plt.title('Top 10 Factors Influencing Graduation', fontsize=18, fontweight='bold')
plt.xlabel('Correlation Strength', fontsize=14)
plt.ylabel('Features', fontsize=14)
plt.tight_layout()
plt.savefig('top_correlations_barh.png', bbox_inches='tight', dpi=150)
plt.show()

Interpretation (Bivariate) - These are the top 10 features most strongly associated with graduation, showing that academic factors dominate, while non-academic influences also contribute.

In [ ]:
# Data Preprocessing - Rwotolara Innocent
# Remove duplicates
df = df.drop_duplicates()
print('Shape after removing duplicates:', df.shape)

# Handle outliers in key numerical features
for col in ['Age at enrollment', 'Admission grade', 'School_Distance_km']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    print(f'Shape after removing outliers in {col}: {df.shape}')

In [ ]:
# Data Preprocessing - Rwotolara Innocent
# Remove duplicates
df = df.drop_duplicates()
print('Shape after removing duplicates:', df.shape)

# Handle outliers in key numerical features
for col in ['Age at enrollment', 'Admission grade', 'School_Distance_km']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    print(f'Shape after removing outliers in {col}: {df.shape}')

In [ ]:
# PCA Implementation - Rwotolara Innocent
# Apply PCA
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)
print('Shape after PCA:', X_pca.shape)
print('Variance retained:', pca.explained_variance_ratio_.sum()).round(3)

In [ ]:
# Plot PCA elbow curve - Rwotolara Innocent
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), np.cumsum(pca.explained_variance_ratio_), marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Elbow Curve')
plt.grid(True)
plt.savefig('pca_elbow_curve.png', bbox_inches='tight', dpi=150)
plt.show()

**Interpretation (PCA)** - The first few components capture most of the important structure in the data, with diminishing returns as more components are added. The “elbow” around 10-12 components suggests an optimal balance, providing most of the information without unnecessary dimensions. By 15-20 components, over 80% of the variance is already explained, so including more offers little additional benefit.

In [ ]:
# PCA Implementation - Rwotolara Innocent
# Feature Importance Ranking After PCA
feature_names = X_encoded.columns.tolist()

feature_importance = np.sum(np.abs(pca.components_), axis=0)

importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": feature_importance
}).sort_values("Importance", ascending=True)

# Plot features ranked by importance
plt.figure(figsize=(10, 8))
plt.barh(importance_df["Feature"], importance_df["Importance"], color="skyblue")
plt.xlabel("Feature Importance", fontsize=12)
plt.title("Original Features Ranked by Importance After PCA", fontsize=14, fontweight="bold")
plt.grid(axis="x", alpha=0.3, linestyle="--")
plt.savefig("pca_feature_importance.png", bbox_inches="tight", dpi=150)
plt.show()

**Interpretation (PCA Feature Importance)** - This plot shows the original features ranked by their contribution to the principal components retained in PCA. Features with higher importance scores have a stronger influence on the transformed PCA space used for modeling. Both academic and non-academic factors contribute, highlighting which variables drive the variance in the dataset and are most influential in predicting student outcomes.

In [ ]:
# Data Preprocessing - Rwotolara Innocent
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42, stratify=y)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Classical ML Models, Hyperparameter Tuning, and Ensemble Methods - Kiwanuka Kenneth
# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

# Top-performing models for tuning
models = {
    'SVM': {'model': SVC(probability=True, random_state=42), 'params': {'C': [0.1, 1, 10], 'kernel': ['rbf', 'poly']}},
    'GradientBoosting': {'model': GradientBoostingClassifier(random_state=42), 'params': {'n_estimators': [100, 200, 300], 'learning_rate': [0.01, 0.05, 0.1]}},
    'Logistic': {'model': LogisticRegression(random_state=42, max_iter=1000), 'params': {'C': [0.1, 1, 10]}},
    'KNN': {'model': KNeighborsClassifier(), 'params': {'n_neighbors': [3, 5], 'weights': ['uniform']}}
}

results = []
tuned_models = {}
best_model = None
best_f1 = 0

In [ ]:
# Tune and train models - Kiwanuka Kenneth
for name, info in models.items():
    print(f'Training and tuning {name}...')
    grid = GridSearchCV(info['model'], info['params'], cv=5, scoring='f1', n_jobs=-1)
    grid.fit(X_train_res, y_train_res)

    tuned_models[name] = grid.best_estimator_
    preds = grid.predict(X_test_scaled)
    probs = grid.predict_proba(X_test_scaled)[:, 1]

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds)
    rec = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    roc = roc_auc_score(y_test, probs)

    results.append({'Model': name, 'Accuracy': acc, 'Precision': prec, 'Recall': rec, 'F1': f1, 'ROC_AUC': roc})

    if f1 > best_f1:
        best_f1 = f1
        best_model = grid.best_estimator_

In [ ]:
# Train stacking model - Kiwanuka Kenneth
estimators = [(name, model) for name, model in tuned_models.items()]
stack = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), cv=5)
stack.fit(X_train_res, y_train_res)

stack_preds = stack.predict(X_test_scaled)
stack_probs = stack.predict_proba(X_test_scaled)[:, 1]

stack_acc = accuracy_score(y_test, stack_preds)
stack_prec = precision_score(y_test, stack_preds)
stack_rec = recall_score(y_test, stack_preds)
stack_f1 = f1_score(y_test, stack_preds)
stack_roc = roc_auc_score(y_test, stack_probs)

results.append({
    'Model': 'Stacking',
    'Accuracy': stack_acc,
    'Precision': stack_prec,
    'Recall': stack_rec,
    'F1': stack_f1,
    'ROC_AUC': stack_roc
})